In [ ]:
#!pip install setuptools==58.0.4
!pip install  -r requirements.txt
!pip install PyMuPDF==1.26.0
!pip install python-docx
!pip install Pillow pytesseract
!pip install jq
!pip install --upgrade --quiet  langchain langchain-community azure-ai-documentintelligence
!pip install semchunk

In [ ]:
import os
import glob
import langchain
from typing import List
import boto3
import ast

#import torch
from dotenv import load_dotenv
from multiprocessing import Pool

import chromadb
from chromadb.config import Settings
from langchain import FAISS
from tqdm import tqdm

from docx import Document as DocxDocument
from pptx import Presentation as PptxDocument

import bs4
from langchain import hub
from langchain_community.document_loaders import UnstructuredFileLoader,UnstructuredWordDocumentLoader,UnstructuredExcelLoader,JSONLoader
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from typing import List, Dict, Any

from langchain.document_loaders import (
    WebBaseLoader,
    CSVLoader,
    EverNoteLoader,
    PDFMinerLoader,
    TextLoader,
    UnstructuredEmailLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredExcelLoader,
    ReadTheDocsLoader,
    Docx2txtLoader
)

from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings   #, OpenAIEmbeddings
from langchain.docstore.document import Document
from dotenv import load_dotenv

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

import semchunk

In [ ]:
import base64
import fitz  
import io
from PIL import Image
from multiprocessing import cpu_count

from typing import List
from langchain_community.document_loaders.base import BaseLoader
from langchain_core.documents import Document

from IPython.display import display, Image
import subprocess
import json
import jq
from functools import partial
import tempfile
import time


In [ ]:
# load_dotenv("config.env")

# # persist_directory = './db'
# embeddings_model_name = "BAAI/bge-base-en-v1.5"
# chunk_size = 1000
# chunk_overlap = 200


# CHROMA_SETTINGS = Settings(
# #        chroma_db_impl='duckdb+parquet',
#         persist_directory=persist_directory,
#         anonymized_telemetry=False
# )

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
def get_key(secret_name,region_name):

    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    key = ast.literal_eval(secret)['key']
    
    return key

openai_api_key=get_key("openai-api-key", "us-west-2")

In [ ]:
# def get_images_from_page(page, scale_factor: float = 1.0, image_encoding: str = "png") -> list[str]:
    
#     mat = fitz.Matrix(scale_factor, scale_factor)
#     pix = page.get_pixmap(matrix=mat)
#     img_bytes = pix.tobytes(output=image_encoding)
    
#     return base64.b64encode(img_bytes).decode("utf-8")

def process_image_with_ai(image_base64: str, model: str) -> str:
      
    max_retries = 5
    attempt = 0
    while attempt < max_retries:
        try:
            client = OpenAI(api_key = get_key("openai-api-key","us-west-2"))
            response = client.chat.completions.create(
                model=model,
                #temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": "Please extract information from the following image in detail and precisely.",
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/png;base64,{image_base64}",
                                },
                            },
                        ],
                    }
                ],
                #max_tokens=1024,
            )
            return response.choices[0].message.content
        
        except Exception as e:
            attempt += 1
            if attempt == max_retries:
                return f"Error: {str(e)}"
            time.sleep(2)  # Wait for 2 seconds before retrying


In [ ]:
def extract_sub_images(document: fitz.Document) -> list[list[dict]]:
    images = []
    for page in document:
        page_images = []
        image_list = page.get_images(full=True)
        for j, img in enumerate(image_list):
            base_image = document.extract_image(img[0])
            page_images.append(
                {
                    "type": "image",
                    "index": j,
                    "base64": base64.b64encode(base_image["image"]).decode("utf-8"),
                    "ext": base_image["ext"],
                }
            )
        images.append(page_images)
    return images


# ## extract information from images only when the page has sub images or tables to avoid processing pure text pages thus reduce processing time
def extract_information_from_page(page: fitz.Page, page_number: int, model: str, source: str, scale_factor: float = 1.5) -> dict:
    text = page.get_text()
    combined_text = [f"Page {page_number + 1} Text:\n{text}\n"]
    document = fitz.open(page.parent.name)
    sub_images = extract_sub_images(document)
    
    # Check if page has tables
    tables = page.find_tables()
    has_tables = len(tables.tables) > 0 if tables else False
    
    # Check if page has sub-images
    has_sub_images = bool(sub_images[page_number])
    
    # Process as image if page has sub-images OR tables
    if has_sub_images or has_tables:
        # Increase resolution using a scale factor=1.5 or 2.0
        mat = fitz.Matrix(scale_factor, scale_factor)
        pix = page.get_pixmap(matrix=mat, alpha=False)
        overview_image_base64 = base64.b64encode(pix.tobytes(output="png")).decode("utf-8")
        
        # Process the page image with AI
        processed_text = process_image_with_ai(overview_image_base64, model)
        combined_text.append(f"Page {page_number + 1} Image Information:\n{processed_text}\n")
        
        result = {
            "text": "\n".join(combined_text),
            "metadata": {
                "overview_image": overview_image_base64,
                "source": source,
                "seq_num": page_number + 1
            }
        }
    else:  # Only plain text on the page
        result = {
            "text": "\n".join(combined_text),
            "metadata": {
                "source": source,
                "seq_num": page_number + 1
            }
        }
    
    return result


# ## extract information from images only when the page has sub images to avoid processing pure text pages thus reduce processing time
# def extract_information_from_page(page: fitz.Page, page_number: int, model: str, source: str, scale_factor: float = 1.5) -> dict:
#     text = page.get_text()
#     combined_text = [f"Page {page_number + 1} Text:\n{text}\n"]

#     document = fitz.open(page.parent.name)
#     sub_images = extract_sub_images(document)
    
#     # Increase resolution using a scale factor=1.5 or 2.0
#     mat = fitz.Matrix(scale_factor, scale_factor)  # Higher scale = better resolution
#     pix = page.get_pixmap(matrix=mat, alpha=False)  # Ensure alpha=False for consistent encoding

#     if sub_images[page_number]:  # Check if there are sub-images on the page
#         overview_image_base64 = base64.b64encode(pix.tobytes(output="png")).decode("utf-8")
#         processed_text = process_image_with_ai(overview_image_base64, model)
#         combined_text.append(f"Page {page_number + 1} Image Information:\n{processed_text}\n")

#         result = {
#             "text": "\n".join(combined_text),
#             "metadata": {
#                 "overview_image": overview_image_base64,
#                 "sub_images": ', '.join([img["base64"] for img in sub_images[page_number]]),
#                 "source": source,
#                 "seq_num": page_number + 1
#             }
#         }
#     else:  # Only text on the page
#         result = {
#             "text": "\n".join(combined_text),
#             "metadata": {
#                 "source": source,
#                 "seq_num": page_number + 1
#             }
#         }
    
#     return result




# ## extract information from images for all pdf page to ensure maxiumn content and slightly different content for same pdf with different filenames
# def extract_information_from_page(page: fitz.Page, page_number: int, model: str, source: str) -> dict:
#     text = page.get_text()
#     combined_text = [f"Page {page_number + 1} Text:\n{text}\n"]

#     document = fitz.open(page.parent.name)
#     sub_images = extract_sub_images(document)

#     overview_image_base64 = base64.b64encode(page.get_pixmap().tobytes(output="png")).decode("utf-8")
#     gpt4_text = process_image_with_gpt4(overview_image_base64, model)
#     combined_text.append(f"Page {page_number + 1} Image Information:\n{gpt4_text}\n")

#     result = {
#         "text": "\n".join(combined_text),
#         "metadata": {
#             "overview_image": overview_image_base64,
#             "sub_images": ', '.join([img["base64"] for img in sub_images[page_number]]),
#             "source": source,
#             "seq_num": page_number + 1
#         }
#     }
    
#     return result



### multi-processing pages in a pdf
def helper(page_number, pdf_path, model):
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_number)
    return extract_information_from_page(page, page_number, model, pdf_path)

def extract_information_from_pdf(pdf_path: str, model: str = "gpt-4o") -> str:
    doc = fitz.open(pdf_path)
    num_pages = len(doc)
    
    args = [(i, pdf_path, model) for i in range(num_pages)]
    
    try:
        with Pool(processes=cpu_count()) as pool:
            results = pool.starmap(helper, args)
    
        extracted_data = [{
            "page_content": result["text"],
            "metadata": result["metadata"]
        } for result in results]
    
        return json.dumps(extracted_data, ensure_ascii=False, indent=4)
    
    except Exception as e:
        print(f"An error occurred during multiprocessing: {e}")
        results = []



In [ ]:
# import fitz
# # Open and save the PDF to potentially fix issues
# pdf_path="datapack-Euhearing_20250817091531.pdf"
# doc = fitz.open(pdf_path)
# doc.save("repaired_datapack-Euhearing_20250817091531.pdf", garbage=4, deflate=True, clean=True)
# doc.close()

In [ ]:
# pdf_path="repaired_datapack-Euhearing_20250817091531.pdf"
# doc = fitz.open(pdf_path)
# text = doc[1].get_text()
# text

In [ ]:
# results=extract_information_from_pdf("datapack-Euhearing_20250817091531.pdf")

# data = json.loads(results)[16:17]

# for page_info in data:
#     # Display the text content
#     print(page_info["page_content"])
    
#     # Display the overview image
#     if 'overview_image' in page_info["metadata"]:
#         print("overview_image:")
#         display(Image(data=base64.b64decode(page_info["metadata"]["overview_image"])))
    
#     # Display sub-images if available
#     if 'sub_images' in page_info["metadata"]:
#         print("sub_images:")
#         sub_image=page_info["metadata"]["sub_images"].split(', ')
#         for img_base64 in sub_image:
#             display(Image(data=base64.b64decode(img_base64)))


In [ ]:
def load_json_data(json_path: str):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [ ]:
##load source files from S3 and save the processed file to S3
def ensure_s3_directory_exists(s3_client, s3_bucket: str, s3_output_dir: str):
    if not s3_output_dir.endswith('/'):
        s3_output_dir += '/'
    try:
        s3_client.put_object(Bucket=s3_bucket, Key=s3_output_dir)
        print(f"Ensured S3 directory s3://{s3_bucket}/{s3_output_dir} exists.")
    except Exception as e:
        print(f"Error ensuring S3 directory exists: {e}")

        
def process_pdfs_in_s3_folder(s3_source_bucket: str, s3_source_folder: str, s3_output_bucket: str, s3_output_folder: str, model: str = "gpt-4o"):
    s3_client = boto3.client('s3')
    
    # Ensure the S3 output directory exists
    ensure_s3_directory_exists(s3_client, s3_output_bucket, s3_output_folder)
    
    response = s3_client.list_objects_v2(Bucket=s3_source_bucket, Prefix=s3_source_folder)
    pdf_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].lower().endswith('.pdf')]

    for pdf_key in pdf_files:
        print(f"Processing s3://{s3_source_bucket}/{pdf_key}")
        output_file_base = os.path.splitext(os.path.basename(pdf_key))[0]
        # Download the PDF file to a temporary file
        temp_pdf_path = f"/tmp/{os.path.basename(pdf_key)}"
        s3_client.download_file(s3_source_bucket, pdf_key, temp_pdf_path)
        
        # Process the PDF
        extracted_info = extract_information_from_pdf(temp_pdf_path, model)
        
        # Check and print the extracted info for debugging
        if extracted_info is None:
            print(f"No information extracted from {pdf_key}.")
            continue
        
        # Save the extracted information to a temporary file
        output_file_name = output_file_base + '.json'
        temp_json_path = f"/tmp/{output_file_name}"
        with open(temp_json_path, 'w', encoding='utf-8') as f:
            f.write(extracted_info)
        
        # Upload the temporary JSON file to S3
        s3_key = os.path.join(s3_output_folder, output_file_name)
        s3_client.upload_file(temp_json_path, s3_output_bucket, s3_key)
        
        # Remove the temporary files
        os.remove(temp_pdf_path)
        os.remove(temp_json_path)
        
        print(f"Saved extracted information to s3://{s3_output_bucket}/{s3_key}")
        

In [ ]:
def process_unprocessed_pdfs_in_s3_folder(s3_source_bucket: str, s3_source_folder: str, s3_output_bucket: str, s3_output_folder: str, model: str = "gpt-4o"):
    s3_client = boto3.client('s3')
    
    # Ensure the S3 output directory exists
    ensure_s3_directory_exists(s3_client, s3_output_bucket, s3_output_folder)
    
    # List all PDF files in the source S3 folder
    response = s3_client.list_objects_v2(Bucket=s3_source_bucket, Prefix=s3_source_folder)
    pdf_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].lower().endswith('.pdf')]
    
    # List all processed JSON files in the output S3 folder
    response = s3_client.list_objects_v2(Bucket=s3_output_bucket, Prefix=s3_output_folder)
    processed_files = {os.path.splitext(os.path.basename(obj['Key']))[0] for obj in response.get('Contents', []) if obj['Key'].endswith('.json')}
    
    for pdf_key in pdf_files:
        output_file_base = os.path.splitext(os.path.basename(pdf_key))[0]
        if output_file_base in processed_files:
            print(f"Skipping already processed file: s3://{s3_source_bucket}/{pdf_key}")
            continue
        
        print(f"Processing s3://{s3_source_bucket}/{pdf_key}")
        
        # Download the PDF file to a temporary file
        temp_pdf_path = f"/tmp/{os.path.basename(pdf_key)}"
        s3_client.download_file(s3_source_bucket, pdf_key, temp_pdf_path)
        
        # Process the PDF
        extracted_info = extract_information_from_pdf(temp_pdf_path, model)
        
        # Check and print the extracted info for debugging
        if extracted_info is None:
            print(f"No information extracted from {pdf_key}.")
            continue
        
        # Save the extracted information to a temporary file
        output_file_name = output_file_base + '.json'
        temp_json_path = f"/tmp/{output_file_name}"
        with open(temp_json_path, 'w', encoding='utf-8') as f:
            f.write(extracted_info)
        
        # Upload the temporary JSON file to S3
        s3_key = os.path.join(s3_output_folder, output_file_name)
        s3_client.upload_file(temp_json_path, s3_output_bucket, s3_key)
        
        # Remove the temporary files
        os.remove(temp_pdf_path)
        os.remove(temp_json_path)
        
        print(f"Saved extracted information to s3://{s3_output_bucket}/{s3_key}")


In [ ]:
def copy_files_from_s3_to_s3(source_bucket: str, source_prefix: str, destination_bucket: str, destination_prefix: str):
    """
    Copies all files from source_prefix in source_bucket to destination_prefix in destination_bucket.
    """
    s3_client = boto3.client('s3')

    # List all objects in the source directory
    response = s3_client.list_objects_v2(Bucket=source_bucket, Prefix=source_prefix)
    if 'Contents' not in response:
        print(f"No files found in {source_bucket}/{source_prefix}")
        return

    for obj in response['Contents']:
        source_key = obj['Key']
        destination_key = source_key.replace(source_prefix, destination_prefix, 1)
        
        copy_source = {
            'Bucket': source_bucket,
            'Key': source_key
        }
        
        print(f"Copying {source_bucket}/{source_key} to {destination_bucket}/{destination_key}")
        s3_client.copy(copy_source, destination_bucket, destination_key)

    print(f"Copied files from s3://{source_bucket}/{source_prefix} to s3://{destination_bucket}/{destination_prefix}")



In [ ]:
#Map file extensions to document loaders and their arguments
LOADER_MAPPING = {
    ".csv": (CSVLoader, {}),
    '.eml':(UnstructuredFileLoader,{}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    #".pdf": (PDFMinerLoader, {}),   #UnstructuredFileLoader
    #".pdf": (CustomPDFLoader, {"model": "gpt-4o"}),
    ".txt": (UnstructuredFileLoader, {"encoding": "utf8"}), #TextLoader
    '.xlsx':(UnstructuredExcelLoader,{}),      #UnstructuredFileLoader  #"mode": "elements"
    #'.XLSX':(UnstructuredExcelLoader,{}), 
    '.docx':(Docx2txtLoader,{}),
    #'.DOCX':(Docx2txtLoader,{}),
    #'.pptx':(UnstructuredPowerPointLoader,{}),      
    #'.json':(JSONLoader,{"jq_schema": '.[] | {text: .text, images: .images}', "text_content": False})
    # Add more mappings for other file extensions and loaders as needed
}


In [ ]:
# Initialize the S3 client
s3_client = boto3.client('s3')

    
# def write_to_temp_file(content: bytes, s3_key: str) -> str:
#     # Extract the filename from the S3 key
#     filename = os.path.basename(s3_key)
#     # Create a temporary file in the current working directory with the extracted filename
#     temp_file_path = os.path.join(os.getcwd(), filename)
    
#     # Write the content to the temporary file
#     with open(temp_file_path, 'wb') as temp_file:
#         temp_file.write(content)
    
#     return temp_file_path

def write_to_temp_file(content: bytes, s3_key: str) -> str:
    # Extract the filename from the S3 key
    filename = os.path.basename(s3_key)
    
    # Define the path to the tmp directory within the current working directory
    temp_dir = os.path.join(os.getcwd(), 'tmp')
    
    # Ensure the tmp directory exists
    os.makedirs(temp_dir, exist_ok=True)
    
    # Create the temporary file path in the tmp directory
    temp_file_path = os.path.join(temp_dir, filename)
    
    # Write the content to the temporary file
    with open(temp_file_path, 'wb') as temp_file:
        temp_file.write(content)
    
    return temp_file_path



def load_single_document_from_s3(s3_bucket: str, s3_key: str) -> List[Document]:
    ext = "." + s3_key.rsplit(".", 1)[-1]
    filename = os.path.basename(s3_key)
    
    retries = 5  # retry loading if loading error happened, after 5 attempts, skip the file
    for attempt in range(retries):
        if ext.lower() in LOADER_MAPPING:
            try:
                loader_class, loader_args = LOADER_MAPPING[ext.lower()]
                obj = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
                file_content = obj['Body'].read()
                
                # Write to a temporary file with the same name as in S3
                temp_file_path = write_to_temp_file(file_content, s3_key)
                loader = loader_class(temp_file_path, **loader_args)
                
                # Load the document
                try:
                    documents = loader.load()
                    # Update document metadata to include only the filename
                    for document in documents:
                        document.metadata['source'] = filename
                    return documents
                except Exception as e:
                    print(f"Error loading content from {temp_file_path} using loader: {e}")
                    documents = []
                finally:
                    # Clean up the temporary file
                    os.remove(temp_file_path)
                
            except Exception as e:
                print(f"Error loading {s3_key} from bucket {s3_bucket}: {e}")
                if attempt < retries - 1:
                    print(f"Retrying... ({attempt + 1}/{retries})")
                    continue
                else:
                    print(f"Failed to load {s3_key} after {retries} attempts. Skipping.")
                    return []
        
        elif ext.lower() == '.json':
            try:
                obj = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
                data = json.loads(obj['Body'].read().decode('utf-8'))
                documents = [Document(page_content=item["page_content"], metadata=item["metadata"]) for item in data]                
#                 documents: List[Document] = []
#                 for item in data:
#                     # start from whatever metadata was in the JSON, then override source
#                     meta = item.get("metadata", {}).copy()
#                     meta["source"] = filename
#                     documents.append(Document(page_content=item["page_content"], metadata=meta))                
                return documents
            except Exception as e:
                print(f"Error loading JSON {s3_key} from bucket {s3_bucket}: {e}")
                if attempt < retries - 1:
                    print(f"Retrying... ({attempt + 1}/{retries})")
                    continue
                else:
                    print(f"Failed to load {s3_key} after {retries} attempts. Skipping.")
                    return []
        
        elif ext.lower() == '.pptx':
            try:
                obj = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
                file_content = obj['Body'].read()
                
                # Write to a temporary file with the same name as in S3
                temp_file_path = write_to_temp_file(file_content, s3_key)
                
                # Create the Azure AI Document Intelligence loader
                azure_endpoint = "https://doc-extraction-test.cognitiveservices.azure.com/"
                azure_api_key = get_key("azure-doc-extraction-test-key", "us-west-2")
                
                loader = AzureAIDocumentIntelligenceLoader(
                    api_endpoint=azure_endpoint,
                    api_key=azure_api_key,
                    file_path=temp_file_path,
                    #api_version=azure_api_version,
                    api_model="prebuilt-layout",
                    mode="markdown")
                        
                try:
                    documents = loader.load()
                    # Create new documents with only the source metadata and page content
                    cleaned_documents = []
                    for document in documents:
                        cleaned_document = Document(page_content=document.page_content,
                                                    metadata={"source": filename})
                        cleaned_documents.append(cleaned_document)
                    return cleaned_documents
                except Exception as e:
                    print(f"Error loading content from {temp_file_path} using Azure AI: {e}")
                    documents = []
                finally:
                    os.remove(temp_file_path) 
                    
            except Exception as e:
                print(f"Error loading PPTX {s3_key} from bucket {s3_bucket}: {e}")
                if attempt < retries - 1:
                    print(f"Retrying... ({attempt + 1}/{retries})")
                    continue
                else:
                    print(f"Failed to load {s3_key} after {retries} attempts. Skipping.")
                    return []

        raise ValueError(f"Unsupported file extension '{ext}'")


    
    
def load_document_key(args) -> List[Document]:
    s3_bucket, s3_key = args
    return load_single_document_from_s3(s3_bucket, s3_key)


# def load_documents_from_s3(s3_bucket: str, s3_folder: str, ignored_files: List[str] = []) -> List[Document]:
#     """
#     Loads all documents from the source S3 bucket and folder, ignoring specified files.
#     Handles pagination if there are more than 1,000 files.
#     """
#     s3_keys = []
#     to_load = list(LOADER_MAPPING.keys()) + ['.json','.pptx']
    
#     # Pagination variables
#     continuation_token = None
#     while True:
#         if continuation_token:
#             response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=s3_folder, ContinuationToken=continuation_token)
#         else:
#             response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=s3_folder)

#         if 'Contents' in response:
#             for obj in response['Contents']:
#                 if any(obj['Key'].lower().endswith(ext) for ext in to_load) and obj['Key'] not in ignored_files:
#                     s3_keys.append(obj['Key'])

#         # If there's a continuation token, it means there are more files to retrieve
#         continuation_token = response.get('NextContinuationToken')
#         if not continuation_token:
#             break
    
#     # Process the files using multiprocessing
#     with Pool(processes=os.cpu_count()) as pool:
#         results = []
#         with tqdm(total=len(s3_keys), desc='Loading new documents', ncols=80) as pbar:
#             for i, docs in enumerate(pool.imap_unordered(load_document_key, [(s3_bucket, key) for key in s3_keys])):
#                 results.extend(docs)
#                 pbar.update()
    
#     return results




def load_documents_from_s3(s3_bucket: str, s3_folder: str, ignored_files: List[str] = []) -> List[Document]:
    """
    Loads all documents from the source S3 bucket and folder, ignoring specified files.
    Uses a paginator to handle buckets with more than 1,000 objects.
    """
    # extensions we care about
    to_load = [ext.lower() for ext in LOADER_MAPPING.keys()] + ['.json', '.pptx']

    # collect all matching keys
    s3_keys: List[str] = []
    paginator = s3_client.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=s3_bucket, Prefix=s3_folder):
        for obj in page.get('Contents', []):
            key = obj['Key']
            key_lower = key.lower()
            if any(key_lower.endswith(ext) for ext in to_load) and key not in ignored_files:
                s3_keys.append(key)

    print(f"Found {len(s3_keys)} documents to load under s3://{s3_bucket}/{s3_folder}")

    # load them in parallel
    documents: List[Document] = []
    with Pool(processes=os.cpu_count()) as pool:
        with tqdm(total=len(s3_keys), desc="Loading documents", ncols=80) as pbar:
            for docs in pool.imap_unordered(load_document_key, [(s3_bucket, key) for key in s3_keys]):
                documents.extend(docs)
                pbar.update()

    return documents




# def process_documents_from_s3(s3_bucket: str, s3_folder: str, ignored_files: List[str] = []) -> List[Document]:
#     """
#     Load documents from S3 and split in chunks
#     """
#     print(f"Loading documents from s3://{s3_bucket}/{s3_folder}")
#     documents = load_documents_from_s3(s3_bucket, s3_folder, ignored_files)
#     if not documents:
#         print("No new documents to load")
#         exit(0)
#     print(f"Loaded {len(documents)} new documents from s3://{s3_bucket}/{s3_folder}")
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#     texts = text_splitter.split_documents(documents)
#     print(f"Split into {len(texts)} chunks of text (max. {chunk_size} tokens each)")
#     return texts




In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'ic-memo-generation/Zenas/docs_ic_memo/'

# load_single_document_from_s3(s3_bucket,f"{s3_folder}Key_LLM_Documents_Series C Cap Table $200M Raise v8 - FINAL.xlsx")

In [ ]:
##the original Literature_InsightsBiopharma_20240223_4211133.xlsx, Literature_SLE ph3 trials globaldata.xlsx, Literature_Barbara_Copy of ms_drugs v2[61].xls  had some filters in excel, can't be loaded, 
##removed the filter and it can be process

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'ic-memo-generation/Zenas/docs_ic_memo/'

# load_single_document_from_s3(s3_bucket,f"{s3_folder}Literature_InsightsBiopharma_20240223_4211133.xlsx")

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'ic-memo-generation/Obsidian/docs_ic_memo/'

# load_single_document_from_s3(s3_bucket,f"{s3_folder}KOLs_Dr Amir Jazaeri call notes 07.23.2021.docx")

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'ic-memo-generation/Obsidian/docs_ic_memo/'
# documents = load_single_document_from_s3(s3_bucket,f"{s3_folder}KOLs_Dr Amir Jazaeri call notes 07.23.2021.docx")

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# texts = text_splitter.split_documents(documents)
# texts

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'ic-memo-generation/Obsidian/docs_ic_memo/'
# documents = load_single_document_from_s3(s3_bucket,f"{s3_folder}KOLs_Dr Amir Jazaeri call notes 07.23.2021.docx")

# chunker = semchunk.chunkerify('gpt-4', chunk_size=500)

# texts = []
# for doc in documents:
#     chunks = chunker(doc.page_content, overlap=100)
#     texts.extend([Document(page_content=chunk, metadata=doc.metadata) for chunk in chunks])
#     # Add filename to each chunk after splitting

# for chunk in texts:
#     filename = chunk.metadata.get('source', 'unknown_file')
#     chunk.page_content = f"From file: {filename}\n\n{chunk.page_content}"
# texts

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'test_bank/xlsx_loader_test/'
# documents = load_single_document_from_s3(s3_bucket,f"{s3_folder}Mosanna_Internal investment process docs_Mosanna DCF v7.xlsx")

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# texts = text_splitter.split_documents(documents)
# texts

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'test_bank/xlsx_loader_test/'
# documents = load_single_document_from_s3(s3_bucket,f"{s3_folder}Mosanna_Internal investment process docs_Mosanna DCF v7.xlsx")

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# texts = text_splitter.split_documents(documents)
# texts

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'test_bank/xlsx_loader_test/'
# documents = load_single_document_from_s3(s3_bucket,f"{s3_folder}Mosanna_Internal investment process docs_Mosanna DCF v7.xlsx")

# chunker = semchunk.chunkerify('gpt-4', chunk_size=500)

# texts = []
# for doc in documents:
#     chunks = chunker(doc.page_content, overlap=100)
#     texts.extend([Document(page_content=chunk, metadata=doc.metadata) for chunk in chunks])
#     # Add filename to each chunk after splitting

# for chunk in texts:
#     filename = chunk.metadata.get('source', 'unknown_file')
#     chunk.page_content = f"From file: {filename}\n\n{chunk.page_content}"
# texts

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'test_bank/xlsx_loader_test/'
# documents = load_single_document_from_s3(s3_bucket,f"{s3_folder}Mosanna_Internal investment process docs_Mosanna DCF v7.md")

# chunker = semchunk.chunkerify('gpt-4', chunk_size=500)

# texts = []
# for doc in documents:
#     chunks = chunker(doc.page_content, overlap=100)
#     texts.extend([Document(page_content=chunk, metadata=doc.metadata) for chunk in chunks])
#     # Add filename to each chunk after splitting

# for chunk in texts:
#     filename = chunk.metadata.get('source', 'unknown_file')
#     chunk.page_content = f"From file: {filename}\n\n{chunk.page_content}"
# texts

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'ic-memo-generation/Obsidian/docs_ic_memo/'

# documents = load_single_document_from_s3(s3_bucket,f"{s3_folder}KOLs_Dr Amir Jazaeri call notes 07.23.2021.docx")

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# texts = text_splitter.split_documents(documents)

# # Add filename to each chunk after splitting
# for chunk in texts:
#     filename = chunk.metadata.get('source', 'unknown_file')
#     chunk.page_content = f"From file: {filename}\n\n{chunk.page_content}"

# texts

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_folder = 'mini-factsheet-generation/Mosanna'

# load_single_document_from_s3(s3_bucket,f"{s3_folder}/pdf_extraction_with_images_update_metadata/3.1.4.1 AVE0118 Investigator's Brochure 3rd Edition.json")

In [ ]:
# load_single_document('./pdf_extraction_with_images/image_extraction_testoutput.json')

In [ ]:
# documents = load_single_document('./pdf_extraction_with_images/image_extraction_testoutput.json')
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# texts = text_splitter.split_documents(documents)
# texts

In [ ]:
# PDFMinerLoader('./docs_scirhom/2023 Sep_SciRhom_Non-Confidential Deck.pdf').load()

In [ ]:
# load_single_document('./docs_scirhom/2023 Sep_SciRhom_Non-Confidential Deck.pdf')

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_key = "document_search/all_files/Engrail--Board Materials--misc board--Gene TherapyCo BP_draft1.25.pptx"

# load_single_document_from_s3(s3_bucket, s3_key)

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_key = "document_search/Mosanna/Mosanna_Internal investment process docs_Mosanna diligence deck.pptx"

# load_single_document_from_s3(s3_bucket, s3_key)

In [ ]:
# s3_bucket = 'plfs-han-llm-experiment'
# s3_key = "document_search/all_files/Engrail--Board Materials--misc board--Gene TherapyCo BP_draft1.25.pptx"

# s3_client = boto3.client('s3')
# obj = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
# file_content = obj['Body'].read()

# # Write to a temporary file with the same name as in S3
# temp_file_path = write_to_temp_file(file_content, s3_key)

# endpoint = "https://doc-extraction-test.cognitiveservices.azure.com/"
# key = get_key("azure-doc-extraction-test-key", "us-west-2")
# loader = AzureAIDocumentIntelligenceLoader(
#     api_endpoint=endpoint, api_key=key, file_path=temp_file_path, api_model="prebuilt-layout"
# )

# documents = loader.load()
# documents